In [ ]:
#importing Libraries
import pandas as pd
import requests
from IPython.display import JSON
import json
!pip install pymysql
import pandas as pd
import sqlalchemy
import requests
from datetime import datetime, date, timedelta
from pytz import timezone
import re

In [ ]:
#Reading demographical data that before collected with Web Scraping.
cities_df = pd.read_csv('citiess.csv')
cities_df

In [ ]:
#Exclude unneccessary column.
cities_df=cities_df.drop(["Unnamed: 0"], axis=1)

In [ ]:
#Check dataframe
cities_df

In [ ]:
#Check data types of dataframe.
cities_df.info()

In [1]:
#importing SQL key from another file.
from Keys import sql_key as password

In [ ]:
#All neccessary informations for connection of database.
host = 'Your Host Adress Here'
schema = 'Your schema name here' 
user = 'Your user name here'
port = 'Your port number here'
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [ ]:
#Push data to database
cities_df.to_sql('cities',con=con,if_exists='append',index=False)

In [ ]:
#Read data from SQL for check
cities_df = pd.read_sql('cities',con=con)
cities_df

# Airports

In [ ]:
list_for_df_airports = []

for i, city in enumerate(cities_df['city_id']):
    url = "https://aerodatabox.p.rapidapi.com/airports/search/location"

    querystring = {"lat":cities_df.iloc[i]["latitude"],"lon":cities_df.iloc[i]['longitude'],"radiusKm":"100","limit":"10","withFlightInfoOnly":"true"}

    headers = {
        "X-RapidAPI-Key": "Your key here",
        "X-RapidAPI-Host": "Your host here"
    }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    list_for_df_airports.append(response.json()['items'])

In [ ]:
list_for_df_airports

In [ ]:
airports_dict = {
                'icao':[],
                'city_name':[]}


for i in range(len(list_for_df_airports)):
    for j in range(len(list_for_df_airports[i])):
            airports_dict['icao'].append(list_for_df_airports[i][j]['icao'])
            airports_dict['city_name'].append(list_for_df_airports[i][j]['municipalityName'])

In [ ]:
airports_df = pd.DataFrame(airports_dict)

In [ ]:
airports_df

In [ ]:
airports_df = airports_df.merge(cities_df, how='inner', left_on='city_name', right_on='city')[["icao", "city_id"]]

In [ ]:
airports_df

In [ ]:
airports_df.info()

In [ ]:
airports_df.to_sql('airports',con=con,if_exists='append',index=False)

In [ ]:
airports_check = pd.read_sql('airports',con=con)
airports_check

## Weather

In [ ]:
cities_df = pd.read_sql('cities',con=con)
cities_df

In [ ]:
from Keys import OW_API_key as key

In [ ]:
weather_dict = {'city_id':[],
                'time':[],
                'temp':[],
                'outlook':[],
                'wind_speed':[],
                }

for i, city in enumerate(cities_df['city_id']):
    url = f"http://api.openweathermap.org/data/2.5/forecast?q={cities_df.iloc[i]['city']}&appid={key}&units=metric"
    response = requests.get(url)
    weather_json = response.json()

    for time in weather_json['list']:
        weather_dict['city_id'].append(city)
        weather_dict['time'].append(time['dt_txt'])
        weather_dict['temp'].append(time['main']['temp'])
        weather_dict['outlook'].append(time['weather'][0]['main'])
        weather_dict['wind_speed'].append(time['wind']['speed'])
      

In [ ]:
weather_df = pd.DataFrame(weather_dict)

weather_df

In [ ]:
JSON(weather_json)

In [ ]:
weather_df['time'] = pd.to_datetime(weather_df['time'])

In [ ]:
weather_df.info()

In [ ]:
weather_df.to_sql('weather',con=con,if_exists='append',index=False)

In [ ]:
weather_check = pd.read_sql('weather',con=con)
weather_check

## Flights

In [ ]:
from Keys import headers_flights as headers_flight

In [ ]:
querystring = {"withLeg":"false","direction":"Arrival","withCancelled":"false","withCodeshared":"false","withCargo":"false","withPrivate":"false","withLocation":"true"}

headers_flights = {
	"X-RapidAPI-Key": "Your key here",
	"X-RapidAPI-Host": "Your host here"
}


In [ ]:
today = datetime.now().astimezone(timezone('Europe/Istanbul')).date()
tomorrow = (today + timedelta(days=1))    

In [ ]:
tomorrow

In [ ]:
today

In [ ]:
times = [["00:00","11:59"],["12:00","23:59"]]

In [ ]:
icao_list = airports_df['icao'].to_list()

In [ ]:
icao_list

In [ ]:
flights_dict = {'ICAO':[],
                'departure_airport':[],
                'local_time':[]}

for time in times:
    
    for icao in icao_list[0:]:
        url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{icao}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(response)
        flight_json = response.json()
        
        for flight in flight_json['arrivals']:
            flights_dict['ICAO'].append(icao)
            try:
                flights_dict['departure_airport'].append(flight['movement']['airport']['icao'])
            except:
                flights_dict['departure_airport'].append('unknown')
            try:
                flights_dict['local_time'].append(flight['movement']['scheduledTime']["local"])
            except:
                flights_dict['local_time'].append(pd.NaT)

In [ ]:
flights_df = pd.DataFrame(flights_dict)

In [ ]:
flights_df

In [ ]:
flights_df['local_time'] = pd.to_datetime(flights_df['local_time'])

In [ ]:
flights_df.info()

In [ ]:
flights_df.to_sql('flights',con=con,if_exists='append',index=False)

In [ ]:
flights_check = pd.read_sql('flights',con=con)
flights_check